<a href="https://colab.research.google.com/github/Yenaaa/24spring_hss510/blob/main/0522_tutorial_for_BERTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gdown
!pip install bertopic
!pip install sentence-transformers
!pip install umap-learn
!pip install hdbscan
!pip install matplotlib
!pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 20.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 22.3 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 9.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (7

In [ ]:
import gdown
link = "https://docs.google.com/spreadsheets/d/1bxuWgL5aVVOsy4F7-27gyj6d-8F42c-C/edit?usp=drive_link&ouid=114525847722656615125&rtpof=true&sd=true"
id = link.split("/")[-2]
output = "text.xlsx"
gdown.download(f"https://drive.google.com/uc?id={id}", output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1bxuWgL5aVVOsy4F7-27gyj6d-8F42c-C
To: /content/text.xlsx
100%|██████████| 2.82M/2.82M [00:00<00:00, 28.8MB/s]


'text.xlsx'

In [ ]:
import pandas as pd
import re
from bertopic import BERTopic
import matplotlib.pyplot as plt

In [ ]:
file_path = 'text.xlsx'
sheet1 = pd.read_excel(file_path, sheet_name='Comments')
sheet2 = pd.read_excel(file_path, sheet_name='Posts')
years_comments = sheet1['Year']
years_posts = sheet2['Year']

In [ ]:
# bringing 'preped_comment' column
comments = sheet1['preped_comment']

# bringing 'preped_posts' and 'preped_Title' from the 2nd sheet
posts = sheet2['preped_posts']
titles = sheet2['preped_Title']

# defining the function
def remove_links(word_list):
    return [word for word in word_list if not re.match(r'http\S+|www\S+', word)]

# prep
def process_list(series):
    processed_list = []
    for text in series:
        if isinstance(text, str):
            text = eval(text)  # str list to list
        if isinstance(text, list):
            cleaned_text = ' '.join(remove_links(text))
            processed_list.append(cleaned_text)
        else:
            processed_list.append(text)
    return processed_list

# list to sentence
comments_str = process_list(comments)
posts_str = process_list(posts)
titles_str = process_list(titles)


In [ ]:
# integrate all data into list
all_texts = comments_str + posts_str + titles_str
all_years = years_comments.tolist() + years_posts.tolist() + years_posts.tolist()
all_texts[:5]

['',
 '',
 '말 없다 하지 마시다 븅 신 같다 년 네다한거려',
 '바니 걸 옷 입다 이퀄리즘 뭔 상관',
 '남 초 전쟁 그러다 공멸 바라다']

In [ ]:
# BERTopic modeling and training, (mulitilingual model)
topic_model = BERTopic(embedding_model="distiluse-base-multilingual-cased-v1",
                       language="multilingual",
                       top_n_words=10,
                       n_gram_range=(1, 2),
                       calculate_probabilities=True,
                       min_topic_size=30,
                       nr_topics=10,
                       verbose=True)

topics, probabilities = topic_model.fit_transform(all_texts)

2024-05-22 02:07:40,310 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Batches:   0%|          | 0/415 [00:00<?, ?it/s]

2024-05-22 02:08:03,277 - BERTopic - Embedding - Completed ✓
2024-05-22 02:08:03,279 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-22 02:08:38,942 - BERTopic - Dimensionality - Completed ✓
2024-05-22 02:08:38,944 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-05-22 02:08:43,869 - BERTopic - Cluster - Completed ✓
2024-05-22 02:08:43,870 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-22 02:08:44,933 - BERTopic - Representation - Completed ✓
2024-05-22 02:08:44,939 - BERTopic - Topic reduction - Reducing number of topics
2024-05-22 02:08:45,981 - BERTopic - Topic reduction - Reduced number of topics from 68 to 10


In [ ]:
topic_info = topic_model.get_topic_info()
topic_info['Name'] = topic_info['Name'].apply(lambda x: x.replace('_', ' '))

print(topic_info)

   Topic  Count                              Name  \
0     -1   5107                   -1 있다 없다 아니다 같다   
1      0   3653                   0 페미니즘 있다 여성 페미   
2      1   2252                   1 이퀄리즘 알다 가다 쓰다   
3      2   1589                 2 이퀄리즘 돼지 없다 나무위키   
4      3    196            3 존나 미러 여성 부여 실컷 혐오 의미   
5      4    178            4 리스트 이퀄 리스트 이퀄 리스트 이퀄   
6      5    117                 5 서버 모드 테스트 사상 검증   
7      6     89        6 개념 개념 없다 이퀄리즘 개념 슬슬 막댓사수   
8      7     37  7 창조경제 창조경제 경제학 사상 창조경제 이퀄리즘이 사상   
9      8     36              8 구글 검색 구글 검색 두껍다 구글   

                                      Representation  \
0      [있다, 없다, 아니다, 같다, 페미, 이퀄리즘, 여자, 여성, 주다, 페미니즘]   
1      [페미니즘, 있다, 여성, 페미, 아니다, 없다, 이퀄리즘, 평등, 같다, 여자]   
2         [이퀄리즘, 알다, 가다, 쓰다, 좋다, 그거, 있다, 이퀄, 이유, 새끼]   
3  [이퀄리즘, 돼지, 없다, 나무위키, 우리 꺼지다, 폐미 피싸개, 피싸개 돼지, 돼...   
4  [존나, 미러 여성, 부여 실컷, 혐오 의미, 실컷 존나, 존나 미러, 실컷, 의미...   
5  [리스트, 이퀄 리스트, 이퀄, 리스트 이퀄, 페미니스트, 젠더 이퀄, 제목, 젠더...   
6   [서버, 모드, 테스트, 사상 검증,

In [ ]:
# checking the probablity of each topic
print(probabilities)

[[9.99602381e-02 5.95907527e-01 6.04787733e-02 ... 6.20170152e-03
  2.65793660e-03 3.24970432e-03]
 [1.05644714e-01 5.51740531e-01 6.39559032e-02 ... 6.56555165e-03
  2.82967505e-03 3.43547136e-03]
 [1.39712388e-01 3.33966292e-02 1.98042844e-01 ... 5.31192629e-03
  7.13332385e-04 3.16161013e-03]
 ...
 [8.33310674e-02 8.87312225e-02 7.92738601e-02 ... 6.36005821e-03
  6.50389589e-04 2.56228404e-03]
 [8.60191462e-01 1.24862823e-02 3.18737108e-02 ... 2.19469310e-03
  4.10099580e-04 1.31568901e-03]
 [2.83614460e-01 1.70774293e-01 2.08043221e-01 ... 1.67246239e-02
  1.66019500e-03 9.91147275e-03]]


In [ ]:
# Index of the document to analyze
document_index = 0  # Index of the document we want to analyze

# Retrieve the topic probabilities for the specified document
document_probabilities = probabilities[document_index]  # Get the topic probabilities for the document at the specified index

# Print the topic probabilities for the specified document
print(f"Document {document_index} topic probabilities: {document_probabilities}")  # Print the topic probabilities for the document


Document 0 topic probabilities: [0.09996024 0.59590753 0.06047877 0.0060613  0.00328367 0.00671634
 0.0062017  0.00265794 0.0032497 ]


In [ ]:
# topic distribution check by year
topics_over_time = topic_model.topics_over_time(docs=all_texts, topics=topics, timestamps=all_years)
print(topics_over_time)

8it [00:07,  1.08it/s]

    Topic                            Words  Frequency  Timestamp
0      -1            있다, 이퀄리즘, 없다, 여성, 아니다        758       2017
1       0           페미니즘, 있다, 여성, 없다, 이퀄리즘        556       2017
2       1             이퀄리즘, 알다, 가다, 쓰다, 주다        339       2017
3       2          나무위키, 이퀄리즘, 없다, 아니다, 토론        332       2017
4       3   존나, 혐오 의미, 존나 미러, 부여 실컷, 미러 여성         64       2017
..    ...                              ...        ...        ...
65      3       존나, 학교 다니다, 학교, 다니다, 존나 미러          8       2024
66      4      리스트, 이퀄 리스트, 이퀄, 사람, 자기 반페미          3       2024
67      5       테스트, 사상 검증, 검증, 사상, 검증 테스트         66       2024
68      6     개념, 설명, 상반 개념, 자체 보리수, 지니 설명          5       2024
69      8  구글 이퀄리즘, 이퀄리즘 치다, 구글, 치다 있다, 치다          1       2024

[70 rows x 4 columns]


In [ ]:
fig = topic_model.visualize_topics_over_time(topics_over_time)  # visualizaion
fig.show()

In [ ]:
topics_over_time


,Topic,Words,Frequency,Timestamp,Name
0,-1,"있다, 이퀄리즘, 없다, 여성, 아니다",758,2017,-1_있다_없다_아니다_같다
1,0,"페미니즘, 있다, 여성, 없다, 이퀄리즘",556,2017,0_페미니즘_있다_여성_페미
2,1,"이퀄리즘, 알다, 가다, 쓰다, 주다",339,2017,1_이퀄리즘_알다_가다_쓰다
3,2,"나무위키, 이퀄리즘, 없다, 아니다, 토론",332,2017,2_이퀄리즘_돼지_없다_나무위키
4,3,"존나, 혐오 의미, 존나 미러, 부여 실컷, 미러 여성",64,2017,3_존나_미러 여성_부여 실컷_혐오 의미
...,...,...,...,...,...
65,3,"존나, 학교 다니다, 학교, 다니다, 존나 미러",8,2024,3_존나_미러 여성_부여 실컷_혐오 의미
66,4,"리스트, 이퀄 리스트, 이퀄, 사람, 자기 반페미",3,2024,4_리스트_이퀄 리스트_이퀄_리스트 이퀄
67,5,"테스트, 사상 검증, 검증, 사상, 검증 테스트",66,2024,5_서버_모드_테스트_사상 검증
68,6,"개념, 설명, 상반 개념, 자체 보리수, 지니 설명",5,2024,6_개념_개념 없다_이퀄리즘 개념_슬슬 막댓사수


In [ ]:

topics_over_time['Proportion'] = topics_over_time['Frequency'] / topics_over_time['Frequency'].sum()

# Remove -1 topic and only consider valid topics
topics_over_time = topics_over_time[topics_over_time['Topic'] >= 0]


In [ ]:
import plotly.express as px

# Remove -1 topic and only consider valid topics
topics_over_time = topics_over_time[topics_over_time['Topic'] >= 0]

# Plot topic proportions over time using plotly
fig = px.line(topics_over_time, x='Timestamp', y='Proportion', color='Topic',
              title='Topic Proportions Over Time', labels={'Proportion': 'Proportion', 'Timestamp': 'Time'})
fig.show()

In [ ]:
# 시각화
# 1. 주제 분포 시각화
fig = topic_model.visualize_topics()
fig.show()

# 2. 주제 상호작용 시각화
fig = topic_model.visualize_hierarchy()
fig.show()

# 3. 주제간 거리 시각화
fig = topic_model.visualize_barchart()
fig.show()

# 4. 주제 간 관계 시각화
fig = topic_model.visualize_heatmap()
fig.show()
